# Import Data From Kaggle

**Note:** *Download your Api Key from Kaggle to download the file directly in google colab or any cloud resources.*

*Else, download the dataset locallly and upload in google colab or any cloud resources you are using.*

*I'll be using Google Colab for this project.*

In [ ]:
!pip -q install kaggle
!pip -q install python_speech_features

In [ ]:
from google.colab import files

# Upload the API token.
uploaded = files.upload()

# Move the uploaded file to the appropriate location.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d carlthome/gtzan-genre-collection

In [ ]:
!ls -lah

In [ ]:
!unzip -q gtzan-genre-collection.zip

# Import Libraries

In [ ]:
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

from python_speech_features import mfcc
import librosa

import joblib

# Data Loading, Preprocessing and Feature Extraction

In [ ]:
# Function to extract features from audio files
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    with open(file_path, "rb") as file:
        audio, sr = librosa.load(file)
        features = []
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20).T, axis=0)
            features.extend(mfccs)
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sr).T,axis=0)
            features.extend(chroma)
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr).T,axis=0)
            features.extend(mel)
    return features

# Function to load dataset
def load_dataset(data_path):
    labels = []
    features = []
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith(".au"):
                file_path = os.path.join(root, file)
                genre = file.split(".")[0]
                features.append(extract_features(file_path))
                labels.append(genre)
    return np.array(features), np.array(labels)

# Define paths
data_path = "genres"
# Load dataset
features, labels = load_dataset(data_path)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Dataset loaded successfully!")

In [ ]:
print(f"Shape of X_train : {X_train.shape}")
print(f"Shape of y_train : {y_train.shape}")
print(f"Shape of X_test : {X_test.shape}")
print(f"Shape of y_test : {y_test.shape}")
print(f"Sample X_train : {X_train[0]}")
print(f"Sample y_train : {y_train[0]}")
print(f"Sample y_train : {y_train[0]}")
print(f"Checking dataset imbalance : {np.array(np.unique(y_train, return_counts=True)).T}")


In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Development and Testing

In [ ]:
# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB()
}

# Train and evaluate classifiers
trained_models = {}
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train_scaled, y_train)
    trained_models[name] = clf
    y_pred = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    results[name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}

# Convert results to DataFrame for easier plotting
import pandas as pd
results_df = pd.DataFrame(results)
results_df = results_df.round(2)

In [ ]:
# Melt the DataFrame to have a long format suitable for Altair
results_melted = results_df.reset_index().melt(id_vars='index', var_name='Metric', value_name='Value')

# Determine the maximum value across all metrics
max_value = results_melted['Value'].max() * 1.2

# Plotting using Altair
bars = alt.Chart(results_melted).mark_bar().encode(
    x=alt.X('index:N', title='Classifier'),
    y=alt.Y('Value:Q', title='Value', scale=alt.Scale(domain=[0, max_value])),
    color='Metric:N',
    tooltip=['index', 'Metric', 'Value']
).properties(
    width=300,
    height=300
)

# Define text inside the bars
text = bars.mark_text(
    align='center',
    baseline='middle',
    dx=0,  # Nudges text to right so it doesn't appear on top of the bar
    dy=-5,  # Nudges text upward
).encode(
    text='Value:Q'
)

# Combine bars and text
chart = (bars + text)

# Arrange two plots per row
chart.facet(
    column='Metric:N',
    columns=2
).properties(title='Performance Metrics of Different Classifiers').interactive()

# Save the Scalers and Models for Deployment

In [ ]:
# Save scaler
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

# Save trained models
models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

for name, clf in trained_models.items():
    model_filename = f"{name.lower().replace(' ', '_')}_model.pkl"
    joblib.dump(clf, os.path.join(models_dir, model_filename))

print("Scalers and models saved successfully!")